## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-30-13-00-00 +0000,missionlocal,Owl pride,https://missionlocal.org/2025/08/sf-photos-owl...
1,2025-08-30-12-53-43 +0000,nyt,"Behind Thai Premier’s Dismissal, Unraveling of...",https://www.nytimes.com/2025/08/30/world/asia/...
2,2025-08-30-12-53-39 +0000,startribune,Somali leader Omar Jamal detained by ICE in do...,https://www.startribune.com/somali-leader-omar...
3,2025-08-30-12-51-00 +0000,wsj,Risk-Taking Is Back in Debt Markets Too,https://www.wsj.com/finance/investing/risk-tak...
4,2025-08-30-12-45-00 +0000,wsj,Former Ukrainian Parliamentary Leader Shot Dead,https://www.wsj.com/world/former-ukrainian-par...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2328/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
323,trump,52
458,court,17
109,new,14
160,president,11
43,ukraine,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
89,2025-08-30-04-15-00 +0000,wsj,"A federal judge in Washington, D.C., temporari...",https://www.wsj.com/politics/policy/judge-bloc...,173
127,2025-08-29-23-59-00 +0000,wsj,A federal appeals court struck down the Trump ...,https://www.wsj.com/politics/policy/appeals-co...,127
152,2025-08-29-22-04-53 +0000,nypost,Trump’s tariffs ruled illegal in stunning blow...,https://nypost.com/2025/08/29/us-news/trumps-t...,121
149,2025-08-29-22-22-47 +0000,cbc,U.S. appeals court finds Trump tariffs unlawfu...,https://www.cbc.ca/news/world/appeals-court-tr...,119
208,2025-08-29-19-45-00 +0000,wsj,A federal judge signaled that she will move qu...,https://www.wsj.com/politics/policy/judge-weig...,113


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
89,173,2025-08-30-04-15-00 +0000,wsj,"A federal judge in Washington, D.C., temporari...",https://www.wsj.com/politics/policy/judge-bloc...
286,48,2025-08-29-14-03-22 +0000,nypost,California Gov. Newsom says crime is higher in...,https://nypost.com/2025/08/29/us-news/californ...
208,43,2025-08-29-19-45-00 +0000,wsj,A federal judge signaled that she will move qu...,https://www.wsj.com/politics/policy/judge-weig...
127,41,2025-08-29-23-59-00 +0000,wsj,A federal appeals court struck down the Trump ...,https://www.wsj.com/politics/policy/appeals-co...
73,39,2025-08-30-07-22-55 +0000,bbc,US blocks Palestinian leader from attending UN...,https://www.bbc.com/news/articles/cjdym32z9v7o...
144,37,2025-08-29-22-49-38 +0000,nypost,Top Zelensky adviser invites Trump special env...,https://nypost.com/2025/08/29/world-news/top-z...
292,37,2025-08-29-13-25-00 +0000,wsj,Trump revokes former Vice President Kamala Har...,https://www.wsj.com/politics/policy/trump-revo...
218,31,2025-08-29-19-21-21 +0000,nyt,Tesla’s Lawyers File Motion to Throw Out $243 ...,https://www.nytimes.com/2025/08/29/business/te...
112,31,2025-08-30-01-00-00 +0000,wsj,Minnesota Gov. Tim Walz campaigned on the nati...,https://www.wsj.com/politics/elections/we-aske...
76,30,2025-08-30-06-52-08 +0000,nypost,Karen Read tells prosecutors ‘you lost big tim...,https://nypost.com/2025/08/30/us-news/karen-re...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
